In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from datetime import datetime
from sklearn.preprocessing import power_transform

Aplicamos la misma transformacion que el script 'processed_data_Vbaseline.ipynb' referente a la baseline

In [2]:
# Carga de datos
df_ratings = pd.read_parquet('../../data/raw/data_model/ratings_test.parquet')
df_tags = pd.read_parquet('../../data/raw/data_model/tags_test.parquet')

In [5]:
# Nos quedamos con el top 3 tag mas relevante por cada movieId
df_tags = df_tags.groupby('movieId').apply(lambda x: x.nlargest(3, 'relevance')).reset_index(drop=True)

In [6]:
# Tranformamos distribucion relevance a normal, se encuentra sesgada
def distribution_relevance_dfTags(df_tags):
    df_tags_copy = df_tags.copy()
    df_tags_copy['relevance']=power_transform(df_tags_copy[['relevance']], method='box-cox')
    return df_tags_copy

In [7]:
df_tags=distribution_relevance_dfTags(df_tags) # Realizamos transformacion de distribucion relevance

In [8]:
df_fullData = df_ratings.merge(df_tags, how='right', on='movieId')

In [10]:
# Los valores nulos de df_ratings los modificaremos con el valor 0
def imputer_(df_ratings):
    df_ratings_copy = df_ratings.copy()
    
    # Columnas numéricas
    columns_numeric = ['userId', 'rating', 'timestamp']
    imp_numeric = SimpleImputer(strategy="constant", fill_value=0)
    df_ratings_copy[columns_numeric] = imp_numeric.fit_transform(df_ratings_copy[columns_numeric])

    # Columnas categóricas
    columns_categorical = ['title', 'genres']
    imp_categorical = SimpleImputer(strategy="constant", fill_value='unknown')
    df_ratings_copy[columns_categorical] = imp_categorical.fit_transform(df_ratings_copy[columns_categorical])

    return df_ratings_copy

In [11]:
df_ratings= imputer_(df_fullData)# llamada a funcion de limpieza de nulos 

In [13]:
def timestamp_to_datetime_dfRatings(df_ratings):
    df_ratings_copy = df_ratings.copy()
    # Convertimos el timestamp a datetime, luego aplicamos el formato año/mes/dia devolviendo un string
    df_ratings_copy['date_rating'] = df_ratings_copy['timestamp'].apply(lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d'))
    # Transformamos a Date el String
    df_ratings_copy['date_rating'] = df_ratings_copy['date_rating'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
    # Eliminamos variable timestamp
    df_ratings_copy.drop('timestamp', axis=1, inplace=True)

    return df_ratings_copy

In [14]:
df_ratings=timestamp_to_datetime_dfRatings(df_ratings) # Transforma timestamp a datetime (las fechas no valoradas se veran asi:"1970-01-01")

In [15]:
# Transformamos las variables del dataset df_ratings
# date_rating, separamos año,mes y dia en variables independientes
def date_rating_indepenDates(df_ratings):
    df_ratings_copy = df_ratings.copy()
    # Extraemos en variables sepadas el año, mes y dia de date_rating
    df_ratings_copy['year_rate'] = df_ratings_copy['date_rating'].dt.year
    df_ratings_copy['month_rate'] = df_ratings_copy['date_rating'].dt.month
    df_ratings_copy['day_rate'] = df_ratings_copy['date_rating'].dt.day
    # Eliminamos variable timestamp
    df_ratings_copy.drop('date_rating', axis=1, inplace=True)
    return df_ratings_copy

In [16]:
df_ratings=date_rating_indepenDates(df_ratings)# Extraemos fechas independientes

In [17]:
# Transformamos las variables del dataset df_ratings
# title, Extraemos el año de publicacion de cada pelicula en una variable nueva
def launchYear_title_dfRatings(df_ratings):
    df_ratings_copy = df_ratings.copy()
    # Extraemos con expresiones regulares los años del title, rellenamos los nulos con 0 y casteamos a int
    df_ratings_copy['launch_year'] = df_ratings_copy['title'].str.extract(r'\((\d{4})\)', expand=False).fillna('0').astype(int)
    # Eliminamos el año y cualquier contenido entre paréntesis en la columna 'title'
    df_ratings_copy['title'] = df_ratings_copy['title'].apply(lambda x: x.split(' (')[0])
    return df_ratings_copy

In [18]:
df_ratings=launchYear_title_dfRatings(df_ratings) # Extrae año del titulo

In [19]:
# Transformamos las variables del dataset df_ratings
# Convertimos userId a entero
def int_userId_dfRatings(df_ratings):
    df_ratings_copy=df_ratings.copy()
    # Convertir 'userId' a int
    df_ratings_copy['userId'] = pd.to_numeric(df_ratings_copy['userId'], errors='coerce').astype('int64')
    return df_ratings_copy

In [20]:
df_ratings=int_userId_dfRatings(df_ratings)# Convierte userId a entero

In [21]:
# Transformamos las variables del dataset df_ratings
# Extraemos la longitud de cada title y lo añadimos como nueva variable
def lenTitle_dfRatings(df_ratings):
    df_ratings_copy=df_ratings.copy()
    #Extraemos la longitud de cada uno de los titulos
    df_ratings_copy['len_title']=df_ratings_copy['title'].apply(lambda x: len(x))

    return df_ratings_copy

In [22]:
df_ratings=lenTitle_dfRatings(df_ratings) # Extraemos longitud del titulo

In [23]:
# Transformamos las variables del dataset df_ratings
# Extraemos  el genero principal de cada movieId (se considera principal al primero que aparece en genres antes del '|')
def mainGenre_dfRatings(df_ratings):
    df_ratings_copy=df_ratings.copy()
    # Extraemos el primer genero de la columna genres y lo almacenamos en una nueva variable
    df_ratings_copy['main_genre']=df_ratings_copy['genres'].apply(lambda x: x.split('|')[0])
    return df_ratings_copy


In [24]:
df_ratings=mainGenre_dfRatings(df_ratings) #Extraemos genero principal

In [26]:
# Guardamos cambios de datos procesados (esto nos permite un mayor control ante nuevos cambios)
df_ratings.to_parquet('../../data/raw/data_model/dataset_process_testV0.1.parquet', index=False)